In [70]:
from dotenv import load_dotenv
import os
import requests 
import pandas as pd 
import urllib
import datetime

In [14]:
load_dotenv()
API_KEY = os.getenv("RIOT_API_KEY")
def get_puuid(game_name=None, tag_line=None, regions="asia"):
    root_url = f"https://{regions}.api.riotgames.com/"
    endpoint = f"riot/account/v1/accounts/by-riot-id/{urllib.parse.quote(game_name)}/{tag_line}"
    response = requests.get(root_url + endpoint, params={"api_key": API_KEY})
    print(endpoint)
    return response.json()['puuid']
get_puuid(game_name="Begamimi",tag_line="VN2")

riot/account/v1/accounts/by-riot-id/Begamimi/VN2


'7llZp-BrYC8yDTt2Iz24LF-AB7L0sYVyQ3g3V2sRKwhRGdDTuS5DTkQXiP_xxP--2Bs-MyASl9ykCA'

In [171]:
def get_match_history_id(puuid=None, region="sea", count=1):
    root_url = f"https://{region}.api.riotgames.com/"
    endpoint = f"lol/match/v5/matches/by-puuid/{puuid}/ids"
    response = requests.get(root_url + endpoint, params={"api_key": API_KEY, "count": count})
    return response.json()

# https://sea.api.riotgames.com/lol/match/v5/matches/VN2_427606326?api_key=RGAPI-6f5ac205-7339-439a-913a-6be684582d04

def get_match_history_details(match_id=None, region="sea"):
    root_url = f"https://{region}.api.riotgames.com/"
    endpoint = f"lol/match/v5/matches/{match_id}"
    response = requests.get(root_url + endpoint, params={"api_key": API_KEY})
    return response.json()

latest_match_history_id = get_match_history_id("7llZp-BrYC8yDTt2Iz24LF-AB7L0sYVyQ3g3V2sRKwhRGdDTuS5DTkQXiP_xxP--2Bs-MyASl9ykCA")
match_result = get_match_history_details(latest_match_history_id[0])

In [172]:
def get_detail_info_with_id_game_name(match_result, riot_id_game_name):
    res = {}
    for participant in match_result['info']['participants']:
        if participant['riotIdGameName'] == riot_id_game_name:
            res = {
                'champion': participant['championName'],
                'deaths': participant['deaths'],
                'double_kills': participant['doubleKills'],
                'triple_kills': participant['tripleKills'],
                'quadra_kills': participant['quadraKills'],
                'penta_kills': participant['pentaKills'],
                'dragon_kills': participant['dragonKills'],
                'kills': participant['kills'],
                'is_win': participant['win'],
                'longest_time_spent_living': participant['longestTimeSpentLiving'],
                'total_damage_dealt_to_champions': participant['totalDamageDealtToChampions'],
            }
    
    return res
    

In [174]:
get_detail_info_with_id_game_name(match_result,"Begamimi")

{'champion': 'Nasus',
 'deaths': 10,
 'double_kills': 1,
 'triple_kills': 0,
 'quadra_kills': 0,
 'penta_kills': 0,
 'dragon_kills': 0,
 'kills': 10,
 'is_win': True,
 'longest_time_spent_living': 619,
 'total_damage_dealt_to_champions': 41638}

In [176]:
def days_hours_minutes(td):
    return td.days, td.seconds//3600, (td.seconds//60)%60

# Duration
duration = match_result['info']['gameDuration']
hour_minute = str(datetime.timedelta(seconds=duration))

# Time end
game_end_timestamp = match_result['info']['gameEndTimestamp'] / 1000
date_from_timestamp = datetime.datetime.fromtimestamp(game_end_timestamp)
current_date = datetime.datetime.now()
duration = current_date - date_from_timestamp
(hour, minute, second) = days_hours_minutes(duration)

# The result
detail_info = get_detail_info_with_id_game_name(match_result,"Begamimi")

2121